In [1]:
import numpy as np
import pandas as pd
import sys
sys.path.append('../src')
import torch

In [2]:
import os
from PIL import Image

In [3]:
from vin_big_dataset import Vin_big_dataset
from transform_vin_big import vin_big_transform
from model_global import global_model
from model_local import local_model

In [4]:
labels_csv = {'train': "/scratch/scratch6/akansh12/DeepEXrays/physionet.org/files/vindr-cxr/1.0.0/annotations/image_labels_train.csv",
             'test': "/scratch/scratch6/akansh12/DeepEXrays/physionet.org/files/vindr-cxr/1.0.0/annotations/image_labels_test.csv"
             }
data_dir = {'train': "/scratch/scratch6/akansh12/DeepEXrays/data/data_256/train/",
           'test': "/scratch/scratch6/akansh12/DeepEXrays/data/data_256/test/"}

global_labels = ['Pleural effusion', 'Lung tumor', 'Pneumonia', 'Tuberculosis', 'Other diseases', 'No finding']

local_labels = ['Aortic enlargement', 'Atelectasis', 'Calcification', 'Cardiomegaly', 'Clavicle fracture', 
                             'Consolidation', 'Emphysema', 'Enlarged PA', 'ILD', 'Infiltration',
                             'Lung Opacity', 'Lung cavity', 'Lung cyst', 'Mediastinal shift',
                             'Nodule/Mass', 'Pleural effusion', 'Pneumothorax',
                             'Pulmonary fibrosis', 'Rib fracture', 'Other lesion', 'COPD', 'No finding']

In [5]:
train_label = pd.read_csv(labels_csv['train'])
test_label = pd.read_csv(labels_csv['test'])
rnd_img = np.random.choice(os.listdir(data_dir['test']), size = 4)

In [6]:
test_tranform = vin_big_transform(t_type = 'test')

In [7]:
global_model = global_model("/scratch/scratch6/akansh12/DeepEXrays/model/0.927723_.pth")
local_model = local_model("/scratch/scratch6/akansh12/DeepEXrays/local_label/0.894920_.pth")

In [14]:
path2img = os.path.join(data_dir['test'] + 'e0dc2e79105ad93532484e956ef8a71a' + '.png')

In [15]:
with torch.no_grad():
    local_model.eval()
    out = local_model(torch.unsqueeze(test_tranform(Image.open(path2img)), dim = 0)) 

In [27]:
local_labels[np.argmax((out > 0.5).numpy()[0].astype('int'))]

'Pleural effusion'

In [18]:
test_label.set_index('image_id').loc['e0dc2e79105ad93532484e956ef8a71a'][local_labels].values

array([0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0])

In [13]:
test_label

,image_id,Aortic enlargement,Atelectasis,Calcification,Cardiomegaly,Clavicle fracture,Consolidation,Edema,Emphysema,Enlarged PA,...,Pneumothorax,Pulmonary fibrosis,Rib fracture,Other lesion,COPD,Lung tumor,Pneumonia,Tuberculosis,Other diseases,No finding
0,e0dc2e79105ad93532484e956ef8a71a,0,1,1,1,0,0,0,0,0,...,1,0,0,0,0,0,1,0,1,0
1,0aed23e64ebdea798486056b4f174424,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,aa15cfcfca7605465ca0513902738b95,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
3,665c4a6d2693dc0286d65ab479c9b169,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,42da2c134b53cb5594774d3d29faac59,1,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,a039af299f86007d0d77da077a6def9a,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2996,aba3d1f5b1c04236f52a8980929b2cfa,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2997,6d3d6b53f358a983b486e9e03144eb62,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2998,d6678cb7ae39f575d35ab9da6d7cb171,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
for i in range(0,len(data_batch)):
    img = (((data_batch.permute(0,2,3,1)[i].cpu().numpy()*std)+mean)*255).astype('uint8')
    annt_edges = mask[i].cpu().numpy()
#     mask = ndi.binary_fill_holes(annt_edges)
    plt.figure(figsize=(10,15))
    plt.subplot(1, 4, 1)
    plt.imshow(img, cmap="gray")
    plt.subplot(1, 4, 2)
    plt.imshow(annt_edges, cmap="gray")
    plt.title("Original-Annotation")
    plt.subplot(1, 4, 3)
    plt.imshow(out_cut[i][0], cmap="gray")
    plt.title("Predicted")
    plt.subplot(1, 4, 4)
    show_img_mask(img, out_cut[i][0])
    plt.title("Predicted-boundries")
    

In [28]:
len(local_labels)

22

In [33]:
np.unique(pd.read_csv("/scratch/scratch6/akansh12/DeepEXrays/physionet.org/files/vindr-cxr/1.0.0/annotations/annotations_test.csv")['class_name'])

array(['Aortic enlargement', 'Atelectasis', 'Calcification',
       'Cardiomegaly', 'Clavicle fracture', 'Consolidation', 'Emphysema',
       'Enlarged PA', 'ILD', 'Infiltration', 'Lung Opacity',
       'Lung cavity', 'Lung cyst', 'Mediastinal shift', 'No finding',
       'Nodule/Mass', 'Other lesion', 'Pleural effusion',
       'Pleural thickening', 'Pneumothorax', 'Pulmonary fibrosis',
       'Rib fracture'], dtype=object)